# 第7章 函数是一等对象

## 7.1 把函数视为对象

In [1]:
# 创建并测试一个函数
def factorial(n):
    """return n!"""
    return 1 if n < 2 else n * factorial(n - 1)

In [2]:
fact = factorial

In [3]:
fact(5)

120

In [4]:
list(map(factorial, range(11)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

- 将函数赋值给变量，通过变量名调用。
- 把函数作为参数传给`map`函数，返回一个可迭代对象。

## 7.2 高阶函数

- map的替代：使用列表推导式。
- filter的替代：在列表推导式中使用`if`过滤。
- reduce的替代：使用内置函数，比如`sum`、`all`、`any`等。

## 7.3 9种可调用对象

- 用户定义的函数：使用`def`语句或`lambda`表达式创建的函数。
- 内置函数：例如`len`或`time.strftime`。
- 内置方法：例如`dict.get`。
- 方法：在类主体中定义的函数。
- 类：调用类时运行类的`__new__`方法创建一个实例，然后运行`__init__`方法，初始化实例，最后把实例返回给调用方。
- 类的实例：如果定义了`__call__`方法，实例可以作为函数调用。
- 生成器函数：主体中有`yield`关键字的函数或方法，返回一个生成器对象。
- 原生协程函数：使用`async def`定义的函数或方法，返回一个协程对象。
- 异步生成器函数：使用`async def`定义，而且主体中有`yield`关键字的函数或方法，返回一个异步生成器，供`async for`使用。

## 7.4 仅限关键字参数

In [6]:
def tag(name, *content, class_=None, **attrs):
    """生成一个或多个HTML标签"""
    if class_ is not None:
        attrs['class'] = class_
    attr_pairs = (f' {attr}="{value}"' for attr, value
                    in sorted(attrs.items()))
    attr_str = ''.join(attr_pairs)
    if content:
        elements = (f'<{name}{attr_str}>{c}</{name}>'
                    for c in content)
        return '\n'.join(elements)
    else:
        return f'<{name}{attr_str} />'

In [8]:
# 传入单个位置参数
tag('br')

'<br />'

In [9]:
# 第一个参数后面的任意数量的参数被*content捕获，存入一个元组
tag('p', 'hello')

'<p>hello</p>'

In [10]:
# tag函数签名中没有明确指定名称的关键字参数被**attrs捕获，存入一个字典
tag('p', 'hello', id=3)

'<p id="3">hello</p>'

In [12]:
# class_参数智能作为关键字参数传入
print(tag('p', 'hello', 'world', class_='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [13]:
# 第一个位置参数也能作为关键字参数传入
tag(content='testing', name='img')

'<img content="testing" />'

In [14]:
# 加上**之后，字典中的所有项作为参数依次传入
my_tag = {'name': 'img', 'title': 'Sunset Boulevard',
           'src': 'sunset.jpg', 'class': 'framed'}
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

## 7.5 支持函数式编程的包

### 7.5.1 operator模块

In [15]:
# 使用reduce函数和operator.mul函数计算阶乘
from functools import reduce
from operator import mul

def factorial(n):
    return reduce(mul, range(1, n + 1))

In [16]:
# 使用itemgetter排序一个元组列表
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),  
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

from operator import itemgetter

for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

('São Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))


In [17]:
cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'São Paulo')


In [19]:
# 使用attrgetter处理定义的具名元组metro_data
from collections import namedtuple

LatLon = namedtuple('LatLon', 'lat lon')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLon(lat, lon)) 
               for name, cc, pop, (lat, lon) in metro_data]

In [20]:
metro_areas[0]

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLon(lat=35.689722, lon=139.691667))

In [21]:
metro_areas[0].coord.lat

35.689722

In [23]:
from operator import attrgetter

name_lat = attrgetter('name', 'coord.lat')

for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

('São Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [24]:
# 使用methodcaller
from operator import methodcaller

s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [25]:
hyphenate = methodcaller('replace', ' ', '-')
hyphenate(s)

'The-time-has-come'

### 7.5.2 使用functools.partial冻结参数

In [26]:
# 使用partial把一个双参数函数改造成只需要一个参数的可调用对象
from operator import mul
from functools import partial

triple = partial(mul, 3)
triple(7)

21

In [27]:
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

## 7.6 杂谈

- Python是一门函数式语言吗？
    - Guido var Rossum（Python之父）：Python中一切好的功能都是从其他语言中借鉴来的。
    - Python提供一等函数，打开了函数式编程的大门。
    - `map`、`filter`和`reduce`最初目的是为Python增加lambda表达式。
    - 影响Python函数式编程惯用法的原因是缺少尾调用消除（通过末尾递归调用，提高计算函数的内存使用效率）。
    - 从设计上，不管函数式语言的定义如何，Python都不是一门函数式语言，只是从函数式语言中借鉴了一些好的想法。

- 匿名函数的问题：匿名函数嵌套的层级太深，不利于调试和处理错误。